# Laden der Daten
In diesem Notebook werden die Daten eingelesen, welche direkt über die URL zugängig sind. Die Geodaten für das Weg- und Velonetz <br>
in Zürich müssen manuell auf der Open Data Seite runtergeladen und dann über einen CSV-Import geladen werden, nachdem die Datei Entzipped wurde.<br>
Grund hierfür ist die Auswahl des Ausschnitts, in welchem das Velonetz angezeigt werden soll. <br>
Dieser muss auf der Seite ausgewählt werden und es wird dann ein Downlaodlink für 5 Minuten zur Verfügung gestellt.


## Verwendete Links/Quellen

In [ ]:
urls = []
urls.append("https://www.hev-schweiz.ch/vermieten/nebenkostenabrechnungen/sonnenscheindauer")
urls.append("https://www.hev-schweiz.ch/vermieten/statistiken/mietpreise/durchschnittliche-mietpreise")

url_energy = "https://opendata.swiss/de/dataset/median-strompreis-per-kanton"

# Installation notwendiger Bibliotheken

Folgende Bibliothekts-Versionen wurden installiert:
- Python pip Version 24.3.1
- Beautifulsoup4 Version 2.6
- Openpyxl Version 2.0.0

In [ ]:
!pip install --upgrade pip
!pip install beautifulsoup4
!pip install openpyxl

# Module importieren

Folgende Packages waren im Einsatz (mittels "!pip list" ausgegeben):
- beautifulsoup4 4.12.3
- urllib3 2.2.3
- openpyxl 3.1.5
- pandas 2.2.2
- pathlib 1.0.1
- google.colb 1.0.0

In [5]:
from bs4 import BeautifulSoup as Soup
import urllib.request
import openpyxl
import pandas as pd
from pathlib import Path
from google.colab import drive # Library to access shared folder "ie_scripting_datasets" on google drive
import os

# Definition Colab Speicherort und Access

In [ ]:
drive.mount('/content/drive') #will give access to "ie_scripting_datasets" on google drive in folder selection

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funktionen

Der folgende Code stellt eine Funktion für das extrahieren von Tabellendaten bereit. Als Input nimmt diese eine HTML-Tabelle an. Daten in "tr" (Table Row) werden als Datenzeilen abgespeichert. Wird ein "th" (Table Header) erkannt, wird diese separat extrahiert. Die Funktion gibt eine Liste von Listen zurück.

In [ ]:
def tableDataText(table):
    def rowgetDataText(tr, coltag='td'):
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow:
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs:
        rows.append(rowgetDataText(tr, 'td') )
    return rows

Nachfolgend die nächste Funktion, welche die Tabellen herunterlädt und in ein Pandas Dataframe umwandelt. Zur Identifizierung der Tabelle, wird nach dessen Klassennamen gesucht.

In [ ]:
def get_dataframes(url):
  html = urllib.request.urlopen(url).read()
  soup = Soup(html, parser='lxml')
  htmltables = soup.find_all('table', { 'class' : 'contenttable acc_content ce-table' })

  all_dataframes = []
  for table in htmltables:
    table = tableDataText(table)
    df = pd.DataFrame(table)
    all_dataframes.append(df)
  return all_dataframes

Anschliessend werden die Dataframes in eine Excel-Datei geschrieben. Die einzelnen Tabellen werden dabei in ein eigenes Register gespeichert. Damit die Register bei der Sonnenscheindauer das entsprechende Jahr träge (diese Information war nicht im DF vorhanden), wurde kurzerhand eine Variable mit dem Wert 2024 erstellt und dessen Wert für jedes Register um die Zahl 1 reduziert. Diese simple Methodik funktioniert aber nur, wenn keine Lücken in den Daten vorhanden sind.
Das erstellen der Register für die Mietpreise war einfacher, da die Überschrift in den Tabellen vorhanden war und mittels "iloc [0,0]" verwendet werden konnte.

In [ ]:
for url in urls:
  if url == urls[0]:
    filepath = Path('/content/drive/My Drive/ie_scripting_datasets/Sonnenscheindauer.xlsx')
    filepath.parent.mkdir(parents=True, exist_ok=True)
    all_dataframes = get_dataframes(url)
    year = 2024
    with pd.ExcelWriter(filepath) as writer:
      for dataframe in all_dataframes:
        dataframe.to_excel(writer, sheet_name=str(year), index=False, header=False)
        year = year - 1

  else:
    filepath = Path('/content/drive/My Drive/ie_scripting_datasets/Mietpreise.xlsx')
    filepath.parent.mkdir(parents=True, exist_ok=True)
    all_dataframes = get_dataframes(url)
    with pd.ExcelWriter(filepath) as writer:
      for dataframe in all_dataframes:
        dataframe.to_excel(writer, sheet_name=str(dataframe.iloc[0,0]), index=False, header=False)

